In [129]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import os
import pickle
import re
import random

In [130]:
# comments have already been downloaded (using curl)
# and are in html/post_pages/
# there are about 400,000 of them

# we want to get:
# game name
# game id if available
# post author name
# post author id
# post content
# post length
# timestamp
# anything else???

post_dir = 'html/post_pages'

In [131]:
post_list = [x for x in os.listdir(post_dir) if x[-5:] == ".html"]
# post_list.sort()
random.shuffle(post_list)
len(post_list)

98782

In [132]:
pf_df = pd.DataFrame(columns=['user name', 'time stamp', 'span count', 'pee count', 'post div count', 'post div zero length', 'post div zero', 'trail anchor', 'link type', 'title'])

for file in post_list[:10000]:
    # get post id from url
    pid_l = re.findall(r'\d+', file)
    pid = int(pid_l[0])
    
    # make a soup
    this_post = 'html/post_pages/' + file
    with open (this_post, 'r', encoding="ISO-8859-1") as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    
    # get title
    title_txt = soup.title.string
    
    # get span count
    all_spans = soup.find_all('span')
    all_dates = soup.find_all('span', class_="post_date")
    all_authors = soup.find_all('span', class_="post_author")
    all_pees = soup.find_all('p')
    trail_anchor = soup.find_all('a', class_="trail")
    all_post_divs = soup.find_all('div', class_='post_content')
    post_div_count = len(all_post_divs)
    span_count = len(all_spans)
    pee_count = len(all_pees)
    
    if len(trail_anchor) > 0:
        trail_url = trail_anchor[0]['href']
        if trail_url[:8] == 'https://':
            anchor_type = 'https'
        else:
            anchor_type = 'relative'
    else:
        trail_url = '';
        anchor_type = '';
    
    if post_div_count > 0:
        post_div_zero = str(all_post_divs[0])
    else:
        post_div_zero = ''
        
    post_div_zero_length = len(post_div_zero)
    
    if span_count > 6:
        user_name = all_spans[1].string
        # stamp = all_spans[2]['title']
        stamp = all_dates[0]['title']
        
    else:
        user_name = ''
        stamp = ''
    
    pf_df.loc[pid] = [user_name, stamp, span_count, pee_count, post_div_count, post_div_zero_length, post_div_zero, trail_url, anchor_type,title_txt]
    

In [133]:
pf_df.sort_index(inplace=True)
pf_df

,user name,time stamp,span count,pee count,post div count,post div zero length,post div zero,trail anchor,link type,title
6,,,6,0,0,0,,,,itch.io
11,leafo,2015-09-04 10:19:03,24,4,3,1041,"<div class=""post_content""><div class=""post_hea...",/jam/leafo-test-jam,relative,Post by leafo in Here's the first topic - itch.io
17,leafo,2015-09-04 10:22:33,12,1,1,1046,"<div class=""post_content""><div class=""post_hea...",/jam/leafo-test-jam,relative,Post by leafo in Here's the first topic - itch.io
18,,,6,0,0,0,,,,itch.io
22,,,6,0,0,0,,,,itch.io
24,,,6,0,0,0,,,,itch.io
29,Headmade Games,2015-09-04 20:02:46,12,3,1,1430,"<div class=""post_content""><div class=""post_hea...",/jam/britishlibrary,relative,Post by Headmade Games in Hardware Information...
32,,,6,0,0,0,,,,itch.io
37,vine,2015-09-09 01:01:13,8,1,1,760,"<div class=""post_content""><div class=""post_hea...",/jam/agdg-demo-day-4,relative,Post by vine in oh hey we have a community now...
40,,,6,0,0,0,,,,itch.io


In [135]:
pickle_name = 'comments_data.pkl'
csv_name = 'comments_data.csv'

#pf_df.to_pickle(pickle_name)
pf_df.to_csv(csv_name, sep='\t', encoding='utf-8')

# a 10000 line csv is 9.1MB

In [ ]:
def info_from_game_post_file(file):
    # get post id from url
    pid_l = re.findall(r'\d+', file)
    pid = int(pid_l[0])
    
    # div id changes for every file
    div_id = 'post-' + pid
    
    # print(did)
    with open (file, 'r', encoding="ISO-8859-1") as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    
    # print(rt)
    print(len(rt))
    print(file)
    # check to see if it's a post page and not a 404 or whatevs
    title_txt = soup.title.string
    print(title_txt[:7])
    if title_txt[:7] == 'Post by':
        
        # dclass = "post_content"
        # div = soup.find('div', id=div_id)
    
    # dtag = soup.div
    # sdiv = dtag.attrs
    
    # ph_id = "post_header"
    
    # ph_div = soup.find('div', ph_id)
    # ph_div_cs = ph_div.findChildren()
    # ph_div_cs = ph_div.find_all("span")
        ph_div_cs = soup.find_all('span')
    # for child in ph_div_cs:
    #     print(child)
        user_span = str(ph_div_cs[1])
        stamp_span = str(ph_div_cs[2])
        user_name = ph_div_cs[1].getText()
        user_id = ph_div_cs[1]('a')[0]['href']
        stamp = stamp_span[31:-46]
        game_url = soup.findAll('h2')[0]('a')[0]['href']
        rettup = (pid, user_id, user_name, game_url, stamp)
    
        return rettup

In [ ]:
this_post = 'html/post_pages/' + post_list[111]
# print(this_post)
urg = info_from_game_post_file(this_post)
print(urg)

In [ ]:
len(post_list)

In [ ]:
post_list.index('117509.html')

In [ ]:
post_list[2283]

In [ ]:
post_tup_list = list()
for post_file in post_list:
    this_file = 'html/post_pages/' + post_file
    post_tup_list.append(info_from_game_post_file(this_file))

In [ ]:
post_tup_list

In [ ]:
len(post_tup_list)